In [1]:
# pip install git+https://github.com/forrestbao/pyeeg.git

In [2]:
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import normalize

import os
import time

In [3]:
destination_folder = "SlidingWindowCNNData"

In [4]:
bands = [4,8,13,16,30,45] 
window_size = 1024  
step_size = 32  
sample_rate = 128 
User_nums = ['2','4','6']

In [5]:
def sliding_window_fft(user_no, bands, window_size, step_size, sample_rate):
    
    all_features=[]
    
    
    user_folder='../../User'+ str(user_no) + '/'
    
    user_csv= os.listdir(user_folder)
    
    user_csv = user_csv[1:]
    
    user_labels_folder = '../../User'+ str(user_no) + '/' + 'Label/'
    
    user_labels=os.listdir(user_labels_folder)
    
    all_data=[]
    all_labels=[]
    
    for csv in user_csv:
        all_data.append(csv)
    
    for lab in user_labels:
        all_labels.append(lab)
     
    
    
    for i in range(0,len(all_data)):
        
        data = pd.read_csv(user_folder + all_data[i])
        cols = data.columns
        cols = cols[:14]
        labels=pd.read_csv(user_labels_folder + all_labels[i])
        labels=labels.values[0][:2]
        
        
        start = 0;
        
        while start + window_size < data.shape[0]:
            
            temp_array = []
            temp_data = [] 
            
            for i in cols:
                
                X = data[i][start : start + window_size] 
                
                Y = pe.bin_power(X, bands, sample_rate)
                
                temp_data = temp_data + list(Y[0])
                
            temp_array.append(np.array(temp_data))
            temp_array.append(labels)
            
            all_features.append(np.array(temp_array))    
            start = start + step_size
            
    all_features = np.array(all_features)
    
    
    
    np.save('EEGCNNData' + str(user_no),all_features, allow_pickle=True, fix_imports=True)
    
#     file_name = "../../" + destination_folder + "/User" + str(user_no) + '/user_features.pkl'
#     pickle.dump(all_features,open(file_name, "wb"))

In [6]:
# for i in User_nums:
sliding_window_fft(4, bands, window_size, step_size, sample_rate)

In [7]:
x_train = []
y_train = []
x_test = []
y_test = []

In [8]:
 with open('EEGCNNData4.npy', 'rb') as file:
        
        array = np.load(file,allow_pickle=True)
        
        for i in range (0,array.shape[0]):
            if i % 7 == 0:
                x_test.append(array[i][0])
                y_test.append(array[i][1])
            else:
                x_train.append(array[i][0])
                y_train.append(array[i][1])

In [9]:
np.array(x_train).shape

(21000, 70)

In [10]:
np.array(y_train).shape

(21000, 2)

In [11]:
np.array(x_test).shape

(3501, 70)

In [12]:
np.array(y_test).shape

(3501, 2)